In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn import naive_bayes
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import itertools
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
import datetime

In [2]:
#path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [3]:
with open(path+"collisions_no1hot.pkl", 'rb') as infile:
    df_no1hot = pickle.load(infile)

## Sort by date and then create train, val, test split

In [4]:
df_no1hot['year'] = pd.to_datetime(df_no1hot['date_time']).dt.year
df_no1hot = df_no1hot.sort_values('date_time')

In [5]:
view_date = pd.to_datetime(df_no1hot['date_time'])

train_indices = (0, np.sum(view_date < datetime.date(2015,9,12))-1)
val_indices = (train_indices[1]+1,\
               train_indices[1] + \
               np.sum((view_date >= datetime.date(2015,9,12)) & (view_date < datetime.date(2016,7,31))))
test_indices = (val_indices[1]+1,\
               val_indices[1] + np.sum(view_date >= datetime.date(2016,7,31)))

In [6]:
df = df_no1hot.drop(['injured', 'killed'], axis=1)

## Train a multinomial on the categorical variables and a gaussian on the numeric

In [7]:
numeric = df.drop(['injured_or_killed'], axis=1)._get_numeric_data().columns.values

In [8]:
categorical = [i for i in df.columns.values if i not in numeric and i != 'injured_or_killed']

In [9]:
multinomial = naive_bayes.MultinomialNB()
gaussian = naive_bayes.GaussianNB()
model = naive_bayes.GaussianNB()

In [10]:
label_encodings = {}

In [11]:
for column in categorical:
    label_encodings[column] = LabelEncoder()
    label_encodings[column].fit(df[column].fillna('NAN'))
    df[column] = label_encodings[column].transform(df[column].fillna('NAN')) 

In [12]:
X_train = df.drop(['injured_or_killed'], axis=1).iloc[train_indices[0]:train_indices[1]]
y_train = df['injured_or_killed'].iloc[train_indices[0]:train_indices[1]]
X_val = df.drop(['injured_or_killed'], axis=1).iloc[val_indices[0]:val_indices[1]]
y_val = df['injured_or_killed'][val_indices[0]:val_indices[1]]
X_test = df.drop(['injured_or_killed'], axis=1).iloc[test_indices[0]:test_indices[1]]
y_test = df['injured_or_killed'][test_indices[0]:test_indices[1]]

In [13]:
X_train.loc[:, categorical]

,borough,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,wind_dir,conditions
693689,2,9,2,7,6,6,17,1
743899,3,9,9,7,6,6,17,1
693691,3,9,9,9,8,6,17,1
693690,1,9,14,7,6,6,17,1
782283,1,16,1,7,6,6,17,1
693692,0,9,9,7,6,6,17,1
693693,1,9,9,7,6,6,17,1
693694,4,9,9,7,6,6,17,1
693696,2,15,14,7,6,6,17,1
693695,1,14,14,7,6,6,17,1


In [14]:
multinomial.fit(X_train.loc[:, categorical], y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
gaussian.fit(X_train.loc[:, numeric], y_train)

GaussianNB(priors=None)

In [35]:
def generate_pred(X):
    pred_cat = [i[1] for i in multinomial.predict_proba(X.loc[:, categorical])]
    pred_num = [i[1] for i in gaussian.predict_proba(X.loc[:, numeric])]
    pred_final = np.multiply(pred_cat, pred_num)
    
    return pred_final

In [36]:
t_auc = metrics.roc_auc_score(y_train, generate_pred(X_train))
v_auc = metrics.roc_auc_score(y_val, generate_pred(X_val))

print (t_auc, v_auc)

0.682322404653 0.654409005105
